# NLP Assignment 2

About the dataset:

The text files contains information about Cynthia Breazeal, a female scientists. Cynthia's career, early education, reseachers, awards are information available in the text files.

In [1]:
#open text file in read mode
text_file = open("cb1.txt", "r")
text_file2 = open("cb2.txt", "r")

#read whole file to a string
data1 = text_file.read()
data2 = text_file2.read()

#close file
text_file.close()
text_file2.close()

In [2]:
import spacy
from spacy.matcher import Matcher 
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load("en_core_web_md")

doc = nlp(data1)
doc2 = nlp(data2)

C:\Users\jingy\anaconda3\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Using Spacy to display the entities and its type

In [3]:
displacy.render(doc,style="ent",jupyter=True)

In [4]:
displacy.render(doc2,style="ent",jupyter=True)

Different entities have different types

# REBEL to perform NER and ER

In [13]:
#create the folder to store the kb object
save_kb_folder = 'C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/' 

In [14]:
# import libraries

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
import IPython
from pyvis.network import Network

# Load Rebel Model

In [15]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Extract Relations From REBEL Model

In [16]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

### Convert texts into kb and extract entities and relations

In [17]:
def from_text_to_kb(text, article_url, span_length=128, article_title=None,
                    article_publish_date=None, verbose=False, extend_kb=None):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    if(extend_kb is None):
        kb = KB()
    else:
        kb = extend_kb

    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                article_url: {
                    "spans": [spans_boundaries[current_span_index]]
                }
            }
            kb.add_relation(relation, article_title, article_publish_date)
        i += 1

    return kb

# Knowledge Base
contains entities and relations which are contained in dictionaries

In [18]:
class KB():
    def __init__(self):
        self.entities = {} # { entity_title: {...} }
        self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
          # meta: { article_url: { spans: [...] } } ]
        self.sources = {} # { article_url: {...} }

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            article_url = list(r["meta"].keys())[0]
            source_data = kb2.sources[article_url]
            self.add_relation(r, source_data["article_title"],
                              source_data["article_publish_date"])

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations
              if self.are_relations_equal(r2, r)][0]

        # if different article
        article_url = list(r2["meta"].keys())[0]
        if article_url not in r1["meta"]:
            r1["meta"][article_url] = r2["meta"][article_url]

        # if existing article
        else:
            spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                            if span not in r1["meta"][article_url]["spans"]]
            r1["meta"][article_url]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            #page = wikipedia.page(candidate_entity, auto_suggest=False)
            page = wikipedia.page(candidate_entity, auto_suggest=False)
        
            entity_data = {
            "title": (page.title).lower(),
            "url": page.url,
            "summary": page.summary
          }
            return entity_data
        except:
            entity_data = {
            "title": candidate_entity+"",
            "url": "",
            "summary": ""
          }
        return entity_data
        #return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r, article_title, article_publish_date):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # add source if not in kb
        article_url = list(r["meta"].keys())[0]
        if article_url not in self.sources:
            self.sources[article_url] = {
                "article_title": article_title,
                "article_publish_date": article_publish_date
            }

            
        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
        print("Sources:")
        for s in self.sources.items():
            print(f"  {s}")
            
    def relations(self):
        for r in self.relations:
            print(f" {r}")

        

### To load and save kb

In [19]:
import pickle

def save_kb(kb, filename):
    with open(f"{save_kb_folder}{filename}", "wb") as f:
        pickle.dump(kb, f)

def load_kb(filename):
    res = None
    with open(f"{save_kb_folder}{filename}", "rb") as f:
        res = pickle.load(f)
    return res

# Increment kb

In [20]:
filename = "Cynthia Breazeal"
kb = from_text_to_kb(data1, "", verbose=True)
save_kb(kb, filename + ".kb")

Input has 702 tokens
Input has 6 spans
Span boundaries are [[0, 128], [114, 242], [228, 356], [342, 470], [456, 584], [570, 698]]


In [21]:
print("Entities extracted using REBEL on first text file")
print("-------------------------------------------------")
for r in kb.entities:
    print(r)

Entities extracted using REBEL on first text file
-------------------------------------------------
cynthia breazeal
2012
personal assistant robot
university of california, santa barbara
massachusetts institute of technology
mit museum
Kismet
Media Lab
Media Arts and Sciences
living with AI
artificial intelligence
"living with AI"


In [22]:
print("Relations extracted using REBEL on first text file")
print("-------------------------------------------------")
for r in kb.relations:
    print(r)

Relations extracted using REBEL on first text file
-------------------------------------------------
{'head': 'cynthia breazeal', 'type': 'inception', 'tail': '2012', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'founded by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'instance of', 'tail': 'personal assistant robot', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'university of california, santa barbara', 'meta': {'': {'spans': [[114, 242]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'massachusetts institute of technology', 'meta': {'': {'spans': [[114, 242], [570, 698]]}}}
{'head': 'cynthia breazeal', 'type': 'has works in the collection', 'tail': 'mit museum', 'meta': {'': {'spans': [[228, 356]]}}}
{'head': 'Kismet', 'type': 'designed by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[228, 356]]}}}
{'head': 'massachusetts

There are 12 entities and 15 relations extracted

## Incremented kb with second text file using REBEL 

In [24]:
kb = load_kb("Cynthia Breazeal.kb") # load previous stored kb
kb = from_text_to_kb(data2, "", verbose=True, extend_kb=kb)
filename = "Cynthia Breazeal2"
save_kb(kb, filename + ".kb")

Token indices sequence length is longer than the specified maximum sequence length for this model (1029 > 1024). Running this sequence through the model will result in indexing errors


Input has 1029 tokens
Input has 9 spans
Span boundaries are [[0, 128], [112, 240], [224, 352], [336, 464], [448, 576], [560, 688], [672, 800], [784, 912], [896, 1024]]


In [25]:
print("Entities extracted using REBEL after incrementing second text file")
for e in kb.entities:
    print(e)

Entities extracted using REBEL after incrementing second text file
cynthia breazeal
2012
personal assistant robot
university of california, santa barbara
massachusetts institute of technology
mit museum
Kismet
Media Lab
Media Arts and Sciences
living with AI
artificial intelligence
"living with AI"
Cynthia Breazeal's
50 Best Inventions of 2008
time (magazine)
50 Best Inventions
2008
Mabu
Catalia Health
cooper hewitt, smithsonian design museum
smithsonian institution
cyberflora
c-3po
anthony daniels
SARS-CoV2 pandemic
2020
innovators under 35
mit technology review
December 15, 2017
Jibo Inc.
Gilbreth Lectures Award
national academy of engineering
25 Best Inventions of 2017
Best Inventions of 2017
George R. Stibitz Computer & Communications Pioneer Award


In [26]:
print("Relations extracted using REBEL after incrementing second text file")
print("-------------------------------------------------")
for r in kb.relations:
    print(r)

Relations extracted using REBEL after incrementing second text file
-------------------------------------------------
{'head': 'cynthia breazeal', 'type': 'inception', 'tail': '2012', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'founded by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'instance of', 'tail': 'personal assistant robot', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'university of california, santa barbara', 'meta': {'': {'spans': [[114, 242]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'massachusetts institute of technology', 'meta': {'': {'spans': [[114, 242], [570, 698]]}}}
{'head': 'cynthia breazeal', 'type': 'has works in the collection', 'tail': 'mit museum', 'meta': {'': {'spans': [[228, 356]]}}}
{'head': 'Kismet', 'type': 'designed by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[228, 356], [0, 128]]

There are a total of 23 new entities and 23 new relations

# REBEL Knowledge Graph

In [92]:
from pyvis import network as net #pip install pyvis==0.3.1
import networkx as nx

In [132]:
#load saved Named Entities and Relations (in kb file) from the KG_Construction.ipynb Jupyter Notebook
kb = load_kb("C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/Cynthia Breazeal.kb")
kb2 = load_kb("C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/Cynthia Breazeal2.kb")

### Construct Graph

In [111]:
#construct graph from the saved Named Entities and Relations (in kb file)
net = Network(
    directed=True, 
    width="1200px", 
    height="1000px", 
    bgcolor="#FFFFFF",
    notebook=True,
    )
    
    # nodes
color_entity = "#00FF00"
for e in kb.entities:
    #net.add_node(e, shape="circle", color=color_entity)
    net.add_node(e)

#edges
for r in kb.relations:
    net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])

net.repulsion(
    node_distance=230,
    damping=0.01
    )

net.set_edge_smooth('dynamic')
net.show('Cynthia Breazeal.html') #saved the graph html file to the same directory as this Jupyter Notebook

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### Knowledge Graph after including entities and relations from second text file

In [112]:
#construct graph from the saved Named Entities and Relations (in kb file)
net2 = Network(
    directed=True, 
    width="1200px", 
    height="1000px", 
    bgcolor="#FFFFFF",
    notebook=True,
    )
    
    # nodes
color_entity = "#00FF00"
for e in kb2.entities:
    #net.add_node(e, shape="circle", color=color_entity)
    net2.add_node(e)

#edges
for r in kb2.relations:
    net2.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])

net2.repulsion(
    node_distance=230,
        #central_gravity=0.02,
        #spring_length=200,
        #spring_strength=0.05,
    damping=0.01
    )

net2.set_edge_smooth('dynamic')    
net2.show('Cynthia Breazeal2.html') #saved the graph html file to the same directory as this Jupyter Notebook

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# NER and RE using StanfordCoreNLP

I have manually copied and pasted the entities and relations into two text csv files. 

The text files are then converted into dataframes. 

The csv files have three columns, head, type and tail.


In [97]:
import pandas as pd

stanford1 = pd.read_csv("stanford1.csv")
stanford2 = pd.read_csv("stanford2.csv")

In [98]:
stanford1

,head,type,tail
0,Cynthia Breazeal,Origin,American
1,Cynthia Breazeal,title,sientist
2,Cynthia Breazeal,title,entrepeneur
3,Cynthia Breazeal,title,professor
4,Cynthia Breazeal,schools_attended,"University of California , Santa Barbara"
5,Media Arts and Sciences,top_members_employees,Cynthia Breazeal
6,Cynthia Breazeal,employee_or_member_of,MIT
7,Cynthia Breazeal,employee_or_member_of,Media Lab
8,Media Lab,top_members_employees,Cynthia Breazeal
9,Personal Robotics,top_members_employees,Cynthia Breazeal


In [99]:
# Add entities in the format of dictionary to match format of kb
def format_data(ent):
    
    # url and summary leave as blanks as they are not necessary
    entity_data = {
        "title" :ent,
        "url"   :"",
        "summary":""
    }
    return entity_data

In [100]:
# Add entities into the kb
def add_stanford_entity(kb, df):
     # Remove duplicate lists
    entity_list = list(set(pd.concat([df['head'], df['tail']]).tolist()))
    entities = [format_data(ent) for ent in entity_list]
    
    if any(ent is None for ent in entities):
        return
    
    # Add entities into kb 
    for e in entities:
        kb.add_entity(e)

## Using entities and relations from Stanford CoreNLP and extend previous kb

In [101]:
# increment kb from rebel
kb = load_kb("C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/Cynthia Breazeal2.kb")

# convert the data into a dictionary
stanford1_dict = stanford1.to_dict(orient='records')
# add relations into kb from the dictionary
kb.relations.extend(stanford1_dict)

# add entities that are present in the relations
add_stanford_entity(kb, stanford1)

filename = "Cynthia Breazeal_stanford1"
save_kb(kb, filename + ".kb")

In [102]:
print("Entities extracted using StanfordCoreNLP + REBEL on first text file")
print("-------------------------------------------------")
for e in kb.entities:
    print(e)

Entities extracted using StanfordCoreNLP + REBEL on first text file
-------------------------------------------------
cynthia breazeal
2012
personal assistant robot
university of california, santa barbara
massachusetts institute of technology
mit museum
Kismet
Media Lab
Media Arts and Sciences
living with AI
artificial intelligence
"living with AI"
Cynthia Breazeal's
50 Best Inventions of 2008
time (magazine)
50 Best Inventions
2008
Mabu
Catalia Health
cooper hewitt, smithsonian design museum
smithsonian institution
cyberflora
c-3po
anthony daniels
SARS-CoV2 pandemic
2020
innovators under 35
mit technology review
December 15, 2017
Jibo Inc.
Gilbreth Lectures Award
national academy of engineering
25 Best Inventions of 2017
Best Inventions of 2017
George R. Stibitz Computer & Communications Pioneer Award
entrepeneur
American
Personal Robotics
director
MIT Quest for Intelligence
sientist
MIT
Massachusetts Institute of Technology
Cynthia Breazeal
Bachelor
dean
University of California , Sa

In [103]:
print("Relations extracted using StanfordCoreNLP + REBEL on first text file")
print("-------------------------------------------------")
for r in kb.relations:
    print(r)

Relations extracted using StanfordCoreNLP + REBEL on first text file
-------------------------------------------------
{'head': 'cynthia breazeal', 'type': 'inception', 'tail': '2012', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'founded by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'instance of', 'tail': 'personal assistant robot', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'university of california, santa barbara', 'meta': {'': {'spans': [[114, 242]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'massachusetts institute of technology', 'meta': {'': {'spans': [[114, 242], [570, 698]]}}}
{'head': 'cynthia breazeal', 'type': 'has works in the collection', 'tail': 'mit museum', 'meta': {'': {'spans': [[228, 356]]}}}
{'head': 'Kismet', 'type': 'designed by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[228, 356], [0, 128]

There a total of 13 newly extracted entities and 21 new relations

In [104]:
kb = load_kb("C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/Cynthia Breazeal_stanford1.kb")

# convert the data into a dictionary
stanford2_dict = stanford2.to_dict(orient='records')
# add relations into kb from the dictionary
kb.relations.extend(stanford2_dict)

# add entities that are present in the relations
add_stanford_entity(kb, stanford2)

filename = "Cynthia Breazeal_stanford2"
save_kb(kb, filename + ".kb")

In [105]:
print("Entities extracted using REBEL after incrementing with second text file")
print("-------------------------------------------------")
for e in kb.entities:
    print(e)

Entities extracted using REBEL after incrementing with second text file
-------------------------------------------------
cynthia breazeal
2012
personal assistant robot
university of california, santa barbara
massachusetts institute of technology
mit museum
Kismet
Media Lab
Media Arts and Sciences
living with AI
artificial intelligence
"living with AI"
Cynthia Breazeal's
50 Best Inventions of 2008
time (magazine)
50 Best Inventions
2008
Mabu
Catalia Health
cooper hewitt, smithsonian design museum
smithsonian institution
cyberflora
c-3po
anthony daniels
SARS-CoV2 pandemic
2020
innovators under 35
mit technology review
December 15, 2017
Jibo Inc.
Gilbreth Lectures Award
national academy of engineering
25 Best Inventions of 2017
Best Inventions of 2017
George R. Stibitz Computer & Communications Pioneer Award
entrepeneur
American
Personal Robotics
director
MIT Quest for Intelligence
sientist
MIT
Massachusetts Institute of Technology
Cynthia Breazeal
Bachelor
dean
University of California 

In [86]:
print("Relations extracted using REBEL after incrementing with second text file")
print("-------------------------------------------------")
for r in kb.relations:
    print(r)

Relations extracted using REBEL after incrementing with second text file
-------------------------------------------------
{'head': 'cynthia breazeal', 'type': 'inception', 'tail': '2012', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'founded by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'instance of', 'tail': 'personal assistant robot', 'meta': {'': {'spans': [[0, 128]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'university of california, santa barbara', 'meta': {'': {'spans': [[114, 242]]}}}
{'head': 'cynthia breazeal', 'type': 'educated at', 'tail': 'massachusetts institute of technology', 'meta': {'': {'spans': [[114, 242], [570, 698]]}}}
{'head': 'cynthia breazeal', 'type': 'has works in the collection', 'tail': 'mit museum', 'meta': {'': {'spans': [[228, 356]]}}}
{'head': 'Kismet', 'type': 'designed by', 'tail': 'cynthia breazeal', 'meta': {'': {'spans': [[228, 356], [0, 

There are 10 new entities and 10 relations extracted 

# REBEL + StanfordcoreNLP Knowledge Graph

In [106]:
#load saved Named Entities and Relations (in kb file) from the KG_Construction.ipynb Jupyter Notebook
kb3 = load_kb("C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/Cynthia Breazeal_stanford1.kb")
kb4 = load_kb("C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/kb/Cynthia Breazeal_stanford2.kb")

In [113]:
#construct graph from the saved Named Entities and Relations (in kb file)
#construct graph from the saved Named Entities and Relations (in kb file)
net3 = Network(
    directed=True, 
    width="1200px", 
    height="1000px", 
    bgcolor="#FFFFFF",
    notebook=True,
    )
    
# nodes
color_entity = "#00FF00"
for e in kb3.entities:
    net3.add_node(e)

#edges
for r in kb3.relations:
    net3.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])

net3.repulsion(
    node_distance=230,
    damping=0.01
    )

net3.set_edge_smooth('dynamic')   
net3.show('Cynthia Breazeal_Stanford1.html') #saved the graph html file to the same directory as this Jupyter Notebook

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [114]:
#construct graph from the saved Named Entities and Relations (in kb file)
net4 = Network(
    directed=True, 
    width="1200px", 
    height="1000px", 
    bgcolor="#FFFFFF",
    notebook=True,
    )

# nodes
color_entity = "#00FF00"
for e in kb4.entities:
    net4.add_node(e)


#edges
for r in kb4.relations:
    net4.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])


net4.repulsion(
    node_distance=230,
    damping=0.01
    )

net4.set_edge_smooth('dynamic')
net4.show('Cynthia Breazeal_stanford2.html') #saved the graph html file to the same directory as this Jupyter Notebook

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# Save to Neo4j

In [115]:
#save the Knowledge Graph to neo4j
#install neo4j from the anaconda prompt with: pip install neo4j
#study the neo4j with pyhton https://neo4j.com/docs/python-manual/current/
from neo4j import GraphDatabase

def save_kb_to_neo4j(kb, uri, username, password):
    # Connect to Neo4j database
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        # Create entities
        for entity_title, entity_data in kb.entities.items():
            entity = {
                "title": entity_title,
                "url": entity_data.get("url", ""),
                "summary": entity_data.get("summary", "")
            }
            session.run(
                "MERGE (e:Entity {title: $title}) "
                "SET e.url = $url, e.summary = $summary",
                entity
            )

        # Create relations
        for relation in kb.relations:
            head = relation["head"]
            tail = relation["tail"]
            relation_type = relation["type"]
            session.run(
                "MATCH (h:Entity {title: $head}), (t:Entity {title: $tail}) "
                "MERGE (h)-[r:RELATION {type: $type}]->(t)",
                {"head": head, "tail": tail, "type": relation_type}
            )

    # Close the connection
    driver.close()
    print("KB saved to Neo4j successfully.")

In [116]:
neo4j_uri = "bolt://localhost:7687"  
neo4j_username = "majyneo4j"  #the user id created in the Neo4j Desktop
neo4j_password = "majyneo4j444444" #the password created based on the user id

# Save KB to Neo4j
save_kb_to_neo4j(kb4, neo4j_uri, neo4j_username, neo4j_password)

KB saved to Neo4j successfully.


# Question Answering

Download dependencies

In [44]:
#pip install packaging==20.9

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pydantic==1.10.9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


Note: you may need to restart the kernel to use updated packages.

rasa 3.8.0a12 requires prompt-toolkit<3.0.29,>=3.0, but you have prompt-toolkit 3.0.41 which is incompatible.

  Using cached pydantic-1.10.9-cp39-cp39-win_amd64.whl.metadata (149 kB)



Using cached pydantic-1.10.9-cp39-cp39-win_amd64.whl (2.2 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.4.0
    Uninstalling pydantic-2.4.0:
      Successfully uninstalled pydantic-2.4.0


In [5]:
import rasa

# Train the RASA Model

In [6]:
config_path = f"C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/config.yml"
training_data_path = f"C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/nlu.yml"
domain_path = f"C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/domain.yml"
output_path = 'C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/'

# Train the model
model_path = rasa.train(domain=domain_path, config=config_path, training_files=training_data_path, output=output_path)

print("* * * * * * * * * * * *")
print(f"Model trained and saved to {model_path}")

No stories or flows present. Just a Rasa NLU model will be trained.


C:\Users\jingy\AppData\Roaming\Python\Python39\site-packages\rasa\engine\recipes\recipe.py:35: FutureWarning: From Rasa Open Source 4.0.0 onwards it will be required to specify a recipe in your model configuration. Defaulting to recipe 'default.v1'.
  rasa.shared.utils.io.raise_deprecation_warning(
C:\Users\jingy\anaconda3\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\jingy\anaconda3\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


2023-12-19 10:38:48 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_WhitespaceTokenizer0
2023-12-19 10:38:48 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=train_RegexFeaturizer1
2023-12-19 10:38:48 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_RegexFeaturizer1
2023-12-19 10:38:48 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=train_LexicalSyntacticFeaturizer2
2023-12-19 10:38:48 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_LexicalSyntacticFeaturizer2
2023-12-19 10:38:48 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=train_CountVectorsFeaturizer3
2023-12-19 10:38:48 [debug    ] graph.node.loading_component 

In [5]:
#load the trained model using Agent
from rasa.core.agent import Agent

# obtain agent path from file
agent = Agent.load(model_path="C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/nlu-20231219-053324-avocado-integer.tar.gz")

C:\Users\jingy\anaconda3\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\jingy\anaconda3\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=NLUMessageConverter constructor=load kwargs={} node_name=nlu_message_converter
2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=FlowsProvider constructor=load kwargs={} node_name=flows_provider
2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=WhitespaceTokenizer constructor=load kwargs={} node_name=run_WhitespaceTokenizer0
2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=RegexFeaturizer constructor=load kwargs={} node_name=run_RegexFeaturizer1
2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=LexicalSyntacticFeaturizer constructor=load kwargs={} node_name=run_LexicalSyntacticFeaturizer2
2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=CountVectorsFeaturizer constructor=load kwargs={} node_name=run_CountVectorsFeaturizer3
2023-12-19 12:19:47 [debug    ] graph.node.loading_component   clazz=CountVectorsFeaturizer construct

### Train the RASA agent

In [8]:
result = await agent.parse_message(
                message_data="What is Cynthia Breazeal's invention")

#retrieve result from the response
intent = result['intent']['name']
#print(results)
print(f"Predicted Intent: {intent}")

2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=WhitespaceTokenizer fn=process node_name=run_WhitespaceTokenizer0
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=RegexFeaturizer fn=process node_name=run_RegexFeaturizer1
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=LexicalSyntacticFeaturizer fn=process node_name=run_LexicalSyntacticFeaturizer2
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer3
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process node_name=run_DIETClassifier5
2023-12-19 10:39:08 [debug    ] graph

The predicted intent is correct, as the message is enquiring about invention

In [9]:
result = await agent.parse_message(
                message_data='What is your best invention')

#retrieve result from the response
intent = result['intent']['name']
#print(results)
print(f"Predicted Intent: {intent}")

2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=WhitespaceTokenizer fn=process node_name=run_WhitespaceTokenizer0
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=RegexFeaturizer fn=process node_name=run_RegexFeaturizer1
2023-12-19 10:39:08 [debug    ] graph.node.running_component   clazz=LexicalSyntacticFeaturizer fn=process node_name=run_LexicalSyntacticFeaturizer2
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer3
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process node_name=run_DIETClassifier5
2023-12-19 10:39:09 [debug    ] graph

In [10]:
result = await agent.parse_message(
                message_data='Who are you')

#retrieve result from the response
intent = result['intent']['name']
#print(results)
print(f"Predicted Intent: {intent}")

2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=WhitespaceTokenizer fn=process node_name=run_WhitespaceTokenizer0
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=RegexFeaturizer fn=process node_name=run_RegexFeaturizer1
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=LexicalSyntacticFeaturizer fn=process node_name=run_LexicalSyntacticFeaturizer2
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer3
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:09 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process node_name=run_DIETClassifier5
2023-12-19 10:39:09 [debug    ] graph

# RASA and Spacy

Training RASA and using Spacy to identify entities

In [11]:
#integrate rasa with Spacy and train a new model with Named Entity Extraction
config_path_spacy = "C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/config_spacy.yml" #configuration with spacy
training_data_path = f"C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/nlu.yml"
domain_path = "C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/domain.yml"
output_path = 'C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/rasa+spacy/'
# Train the model
model_path = rasa.train(domain=domain_path, config=config_path_spacy, training_files=training_data_path, output=output_path)

print(f"Model trained and saved to {model_path}")

No stories or flows present. Just a Rasa NLU model will be trained.


C:\Users\jingy\AppData\Roaming\Python\Python39\site-packages\rasa\engine\recipes\recipe.py:35: FutureWarning: From Rasa Open Source 4.0.0 onwards it will be required to specify a recipe in your model configuration. Defaulting to recipe 'default.v1'.
  rasa.shared.utils.io.raise_deprecation_warning(


2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=provide_SpacyNLP0
2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_SpacyNLP0
2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_SpacyTokenizer1
2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_SpacyFeaturizer3
2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=train_CountVectorsFeaturizer4
2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent constructor=create kwargs={} node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:16 [debug    ] graph.node.loading_component   clazz=FingerprintComponent const

In [6]:
# agent for RASA + Spacy
agent2 = Agent.load(model_path='C:/Users/jingy/OneDrive/Desktop/xmum_mjy/year 3 sem II/NLP/assignment2/rasa/rasa+spacy/nlu-20231219-053510-intense-mayonnaise.tar.gz')

2023-12-19 12:20:01 [debug    ] graph.node.loading_component   clazz=NLUMessageConverter constructor=load kwargs={} node_name=nlu_message_converter
2023-12-19 12:20:01 [debug    ] graph.node.loading_component   clazz=FlowsProvider constructor=load kwargs={} node_name=flows_provider
2023-12-19 12:20:01 [debug    ] graph.node.loading_component   clazz=SpacyNLP constructor=load kwargs={} node_name=provide_SpacyNLP0
2023-12-19 12:20:02 [debug    ] graph.node.loading_component   clazz=SpacyNLP constructor=load kwargs={} node_name=run_SpacyNLP0
2023-12-19 12:20:04 [debug    ] graph.node.loading_component   clazz=SpacyTokenizer constructor=load kwargs={} node_name=run_SpacyTokenizer1
2023-12-19 12:20:04 [debug    ] graph.node.loading_component   clazz=SpacyEntityExtractor constructor=load kwargs={} node_name=run_SpacyEntityExtractor2
2023-12-19 12:20:04 [debug    ] graph.node.loading_component   clazz=SpacyFeaturizer constructor=load kwargs={} node_name=run_SpacyFeaturizer3
2023-12-19 12:20:0

C:\Users\jingy\AppData\Roaming\Python\Python39\site-packages\rasa\utils\train_utils.py:525: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss.
  rasa.shared.utils.io.raise_warning(


2023-12-19 12:20:12 [debug    ] graph.node.loading_component   clazz=RegexMessageHandler constructor=load kwargs={} node_name=run_RegexMessageHandler


In [13]:
#Perform intent classification using the agent parse_message
await agent2.parse_message( message_data='Can you tell me who are you?')

2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

{'text': 'Can you tell me who are you?',
 'intent': {'name': 'introduction', 'confidence': 0.999524712562561},
 'entities': [],
 'text_tokens': [(0, 3),
  (4, 7),
  (8, 12),
  (13, 15),
  (16, 19),
  (20, 23),
  (24, 27),
  (27, 28)],
 'intent_ranking': [{'name': 'introduction', 'confidence': 0.999524712562561},
  {'name': 'career', 'confidence': 0.0003980153414886445},
  {'name': 'study', 'confidence': 7.122199167497456e-05},
  {'name': 'award', 'confidence': 3.257959861002746e-06},
  {'name': 'invention', 'confidence': 2.8205456601426704e-06}]}

In [14]:
#Perform intent classification using the agent parse_message
await agent2.parse_message( message_data='Are you Cynthia Breazel?')

2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

C:\Users\jingy\AppData\Roaming\Python\Python39\site-packages\rasa\shared\utils\io.py:99: UserWarning: Parsed an entity 'PERSON' which is not defined in the domain. Please make sure all entities are listed in the domain.
  More info at https://rasa.com/docs/rasa/domain


{'text': 'Are you Cynthia Breazel?',
 'intent': {'name': 'introduction', 'confidence': 0.9975766539573669},
 'entities': [{'entity': 'PERSON',
   'value': 'Cynthia Breazel',
   'start': 8,
   'confidence': None,
   'end': 23,
   'extractor': 'SpacyEntityExtractor'}],
 'text_tokens': [(0, 3), (4, 7), (8, 15), (16, 23), (23, 24)],
 'intent_ranking': [{'name': 'introduction', 'confidence': 0.9975766539573669},
  {'name': 'study', 'confidence': 0.0018980924505740404},
  {'name': 'career', 'confidence': 0.00028909361572004855},
  {'name': 'invention', 'confidence': 0.00020933257474098355},
  {'name': 'award', 'confidence': 2.6825735403690487e-05}]}

One entity is extracted, Cynthia Breazeal (PERSON)

In [15]:
#Perform intent classification using the agent parse_message
await agent2.parse_message( message_data='Did Cynthia Breazeal study at MIT?')

2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

{'text': 'Did Cynthia Breazeal study at MIT?',
 'intent': {'name': 'study', 'confidence': 0.9998600482940674},
 'entities': [{'entity': 'PERSON',
   'value': 'Cynthia Breazeal',
   'start': 4,
   'confidence': None,
   'end': 20,
   'extractor': 'SpacyEntityExtractor'}],
 'text_tokens': [(0, 3),
  (4, 11),
  (12, 20),
  (21, 26),
  (27, 29),
  (30, 33),
  (33, 34)],
 'intent_ranking': [{'name': 'study', 'confidence': 0.9998600482940674},
  {'name': 'career', 'confidence': 7.842678314773366e-05},
  {'name': 'introduction', 'confidence': 4.2062347347382456e-05},
  {'name': 'invention', 'confidence': 1.001271630229894e-05},
  {'name': 'award', 'confidence': 9.459186912863515e-06}]}

One entities is extracted, Cynthia Breazeal (PERSON) 

In [16]:
#Perform intent classification using the agent parse_message
await agent2.parse_message( message_data='Did Cynthia Breazeal invent Learnado and Jibo')

2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 10:39:31 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

{'text': 'Did Cynthia Breazeal invent Learnado and Jibo',
 'intent': {'name': 'invention', 'confidence': 0.992759108543396},
 'entities': [{'entity': 'PERSON',
   'value': 'Cynthia Breazeal',
   'start': 4,
   'confidence': None,
   'end': 20,
   'extractor': 'SpacyEntityExtractor'},
  {'entity': 'PERSON',
   'value': 'Learnado',
   'start': 28,
   'confidence': None,
   'end': 36,
   'extractor': 'SpacyEntityExtractor'},
  {'entity': 'PERSON',
   'value': 'Jibo',
   'start': 41,
   'confidence': None,
   'end': 45,
   'extractor': 'SpacyEntityExtractor'}],
 'text_tokens': [(0, 3),
  (4, 11),
  (12, 20),
  (21, 27),
  (28, 36),
  (37, 40),
  (41, 45)],
 'intent_ranking': [{'name': 'invention', 'confidence': 0.992759108543396},
  {'name': 'career', 'confidence': 0.005635673180222511},
  {'name': 'study', 'confidence': 0.001261723111383617},
  {'name': 'award', 'confidence': 0.0002824667899403721},
  {'name': 'introduction', 'confidence': 6.106740329414606e-05}]}

The agent indentified 3 entities, Cynthia Breazeal (PERSON), Learnado (PERSON) and Jibo (PERSON). The predicted intent is correct.

In [17]:
#Perform intent classification using the agent parse_message
await agent2.parse_message( message_data='Cynthia Breazeal studied at The University of California, Santa Barbara')

2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 10:39:32 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

C:\Users\jingy\AppData\Roaming\Python\Python39\site-packages\rasa\shared\utils\io.py:99: UserWarning: Parsed an entity 'GPE' which is not defined in the domain. Please make sure all entities are listed in the domain.
  More info at https://rasa.com/docs/rasa/domain


{'text': 'Cynthia Breazeal studied at The University of California, Santa Barbara',
 'intent': {'name': 'career', 'confidence': 0.5326156616210938},
 'entities': [{'entity': 'PERSON',
   'value': 'Cynthia Breazeal',
   'start': 0,
   'confidence': None,
   'end': 16,
   'extractor': 'SpacyEntityExtractor'},
  {'entity': 'GPE',
   'value': 'Santa Barbara',
   'start': 58,
   'confidence': None,
   'end': 71,
   'extractor': 'SpacyEntityExtractor'}],
 'text_tokens': [(0, 7),
  (8, 16),
  (17, 24),
  (25, 27),
  (28, 31),
  (32, 42),
  (43, 45),
  (46, 56),
  (56, 57),
  (58, 63),
  (64, 71)],
 'intent_ranking': [{'name': 'career', 'confidence': 0.5326156616210938},
  {'name': 'study', 'confidence': 0.45336779952049255},
  {'name': 'invention', 'confidence': 0.009126434102654457},
  {'name': 'introduction', 'confidence': 0.004122691694647074},
  {'name': 'award', 'confidence': 0.0007674255757592618}]}

The agent indentified 2 entity, Cynthia Breazeal (PERSON) and Data

# Answer Retrieval

In [10]:
nlp = spacy.load("en_core_web_sm")

async def get_entities_intent(ques):
    result = await agent2.parse_message(message_data=ques)
    
    # retrieve entities
    doc = nlp(ques)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
        
    #retrieve result from the response
    intent = result['intent']['name']

    return intent, entities

In [11]:
question = "What award did Cynthia Breazeal win?"
intent, entities = await get_entities_intent(question)

2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 12:20:28 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

In [12]:
print(intent)
print(entities)

award
[('Cynthia Breazeal', 'PERSON')]


In [13]:
from neo4j import GraphDatabase

# neo4j driver 
neo4j_uri = "bolt://localhost:7687"
neo4j_user = "majyneo4j"
neo4j_password = "majyneo4j444444"

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [51]:
def award_intent_question(entities, ques):
    if not entities:
        print("There are no entities")
    
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]
    
    # store responses
    responses = set()
    
    with driver.session() as session:
        # obtain entities which have relation of "award received" with entity
        if ((entity_labels[0] == 'PERSON') and ('win' in ques or 'award' in ques or 'get' in ques)):
            result = session.run("MATCH (p1:Entity {title: $title}) - [R:RELATION {type: $relation_type}] -
                                 (p2:Entity) RETURN p2.title AS n",
                                 title=entity_titles[0], relation_type = "award received")
           
        # add the entity extracted into responses
            for record in result:
                output = record["n"]
                if output is not None:
                    responses.add(''.join(output))
                    
        if responses:
            return "\n".join(responses)
        else:
            return f"No award information related to {entity_titles[0]}."

In [33]:
def study_intent_question(entities, ques):
    if not entities:
        print("There are no entities")
    
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]
    
    responses = set()
    
    with driver.session() as session:
        # obtain entities which have relation of "educated at" with entity
        if ((entity_labels[0] == 'PERSON') and ('study' in ques or 'educated' in ques or 'university' in ques)):
            result = session.run("MATCH (p1:Entity {title: $title}) - [R:RELATION {type: $relation_type}] - 
                                 (p2:Entity) RETURN p2.title AS n",
                               title=entity_titles[0], relation_type="educated at")
            
            # add the entity extracted into responses
            for record in result:
                location_title = record["n"]

                if location_title is not None:
                    responses.add(''.join(location_title))
            
        if responses:
            return "\n".join(responses)
        else:
            return f"No study information related to {entity_titles[0]}."

In [54]:
def career_intent_question(entities, ques):
    if not entities:
        print("There are no entities")
    
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]
    
    responses = set()
    
    with driver.session() as session:
        # obtain entities which have relation of "title" and "employee_or_member_of" with entity
        if ((entity_labels[0] == 'PERSON') and ('work' in ques or 'profession' in ques)):
            result = session.run("MATCH (p1:Entity {title: $title}) - [R:RELATION {type: $relation_type}] - 
                                 (p2:Entity) RETURN p2.title AS n",
                                 title=entity_titles[0], relation_type="title")
            result2 = session.run("MATCH (p1:Entity {title: $title}) - [R:RELATION {type: $relation_type}] - 
                                  (p2:Entity) RETURN p2.title AS n",
                                 title=entity_titles[0], relation_type="employee_or_member_of")
            
            # add the entity extracted into responses
            for record in result:
                output = record["n"]
                if output is not None:
                    responses.add(''.join(output))
                    
            for record2 in result2:
                output = record['n']
                if output is not None:
                    responses.add(''.join(output))
                    
        if responses:
            return "\n".join(responses)
        else:
            return f"No career information related to {entity_titles[0]}."

In [17]:
# have three functions for 3 intents
async def answer_retrieval(ques):
    intent, entities = await get_entities_intent(ques)
    
    if intent == 'award':
        ans = award_intent_question(entities, ques)
    elif intent == 'study':
        ans = study_intent_question(entities, ques)
    elif intent == 'career':
        ans = career_intent_question(entities, ques)

    else:
        ans = "Huh, question unavailable to answer."
        
    print("Question :\n", ques)
    print("Ansewer :\n", ans)

In [ ]:
state_list = ['Penang', 'Sabah', 'Kuala Lumpur']
preference_list = ['price', 'service', 'atmosphere', 'food', 'cleanliness', 'location']
type_list = ['local', 'chinese', 'indian', 'thai', 'dessert', 'western', 'korean', 'japanese']

# Retrieving Information using answer_retrieval

In [52]:
question = "What award did Cynthia Breazeal win?"
await answer_retrieval(question)

2023-12-19 12:48:24 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 12:48:24 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 12:48:24 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 12:48:25 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 12:48:25 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 12:48:25 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 12:48:25 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 12:48:25 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

In [49]:
question = "Where did Cynthia Breazeal study at?"
await answer_retrieval(question)

2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 12:43:47 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

In [57]:
question = "What does Cynthia Breazeal work as?"
await answer_retrieval(question)

2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 12:50:32 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

In [27]:
question = "Cynthia Breazeal profession"
await answer_retrieval(question)

2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-19 12:34:17 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

Successfully retrieved information regarding intent of career, study and award from knowledge graph in Neo4j.